# Atelier 5, PageRank et PageRank personnalisé #
Cet atelier porte sur l'algorithme PageRank et sa version personnalisée personnalisé pour effectuer des recommandations.  Utilisez la bibliothèque "igraph" pour cet exercice et les données de MovieLens 100k.

Chaque question compte pour 5 points.

## Question 1 ##
(1) Créez la matrice d'adjacence du graphe biparti que nous nommerons B.  Cette matrice doit avoir une dimension de (943+1682,943+1682).  Vous pouvez la créer à partir de la matrice d'adjacence des votes de dimension (943,1682).

In [28]:
import warnings

warnings.filterwarnings("ignore")
import pandas as pd
import numpy as np
from tqdm.autonotebook import tqdm
import igraph as ig
import matplotlib.pyplot as plt

u_data = pd.read_csv("data/votes.csv")
u_user = pd.read_csv("data/u.csv")
u_item = pd.read_csv("data/items.csv")

# Matrice d'adjacence
m = u_data.pivot(index="user.id", columns="item.id", values="rating").to_numpy(na_value=0)
B = np.zeros((m.shape[0]+m.shape[1], m.shape[0]+ m.shape[1]))
B[:m.shape[0], :m.shape[1]] = m
B[m.shape[0]:, m.shape[1]:] = m.T
print(f"Matrice d'adjacence bipartie: {B.shape}")
B

Matrice d'adjacence bipartie: (2625, 2625)


array([[5., 3., 4., ..., 0., 0., 0.],
       [4., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

## Question 2 ##
(2) Calculez le vecteur du PageRank de la matrice biparti B.

In [34]:
d = 0.85
n = B.shape[1]
d_matrix = (1-(np.ones(n)*d))/n
s = np.sum(B, axis=0)
s[s == 0] = 1 # Avoid division by zero

r = np.ones(n) / n
r_prev = np.zeros(n)
while np.linalg.norm(r - r_prev) > 1e-10:
    r_prev = r
    r = d_matrix + (d * B.T) @ (r / s)
r

array([0.00908438, 0.00219047, 0.00133335, ..., 0.00068453, 0.00308071,
       0.00533969])

## Question 3 ##
(3) Les titres ci-dessous représentent les films de deux réalisateurs, Alfred Hitchcock et Stanley Kubrick, que l'on retrouve dans les données Movielens.  Utilisez la version personnalisée de l'algorithme PageRank et présumez un utilisateur qui aime les deux premiers de chaque liste (Birds, Vertigo, Full Metal [...] et Clockwork [...]), mais qui n'a pas de vote pour les autres.  Quels sont les 80 premier films qui seraient recommandés et comment cela se compare-t-il avec la version non personnalisée?

In [5]:
hitchcock_titres = [
    "Birds, The (1963)",
    "Vertigo (1958)",
    "North by Northwest (1959)",
    "Rear Window (1954)",
    "Dial M for Murder (1954)",
    "Psycho (1960)",
]

kubrick_titres = [
    "Full Metal Jacket (1987)",
    "Clockwork Orange, A (1971)",
    "2001: A Space Odyssey (1968)",
    "Dr. Strangelove or: How I Learned to Stop Worrying and Love the Bomb (1963)",
]

## Question 4 ##
(4) Comment pourrait-on intégrer à cet algorithme l'information du réalisateur de ces films? Quel est le résultat par rapport à (3)?